In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [15]:
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [16]:
training_labels = training_labels[['person','label']]
training_labels['label'].value_counts()

0    18434
1      980
Name: label, dtype: int64

In [17]:
data.head()

,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,maximo,minimo,suma_resolution,promedio_resolution,maximo_resolution,minimo_resolution,promedio de TFIdf modelo,Suma de TFidf modelo,Maximo de TFIdf modelo,Minimo de TFIdf modelo
0,0008ed71,2.0,0.0,0.0,0.0,0.0,0.0,0,0,3,...,20180517162837,20180517122747,4147200,691200.000000,2073600,0,1.617163,4.851488,1.859879,1.300103
1,00091926,1.0,0.0,25.0,0.0,0.0,0.0,15,25,2,...,20180531195203,20180503220829,26738688,59684.571429,786432,0,1.490856,557.580165,2.209395,1.000000
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0,...,20180326145658,20180326145111,230400,23040.000000,230400,0,1.219303,3.657909,1.357806,1.000000
3,000ba417,0.0,0.0,6.0,0.0,0.0,0.0,1,24,6,...,20180526130922,20180517111145,4718592,22905.786408,786432,0,1.798317,287.730661,2.299973,1.000000
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,20180529003807,20180529002747,230400,13552.941176,230400,0,1.000000,4.000000,1.000000,1.000000


In [18]:
features = pd.merge(training_labels, data, on='person', how='inner')
features = features.fillna(0)
features['label'].value_counts()

0    18434
1      980
Name: label, dtype: int64

In [19]:
labels = features['label']
features = features.drop(['label','person'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  14560 Test:  4854


In [20]:
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]

In [21]:
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

In [24]:
import catboost as cb
from catboost import CatBoostClassifier

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300],
         'scale_pos_weight': [18]}
cb = CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(x_train, y_train)

0:	learn: 0.6599524	total: 80.9ms	remaining: 24.2s
1:	learn: 0.6331181	total: 159ms	remaining: 23.7s
2:	learn: 0.6131228	total: 236ms	remaining: 23.3s
3:	learn: 0.5871964	total: 324ms	remaining: 24s
4:	learn: 0.5702547	total: 405ms	remaining: 23.9s
5:	learn: 0.5549532	total: 482ms	remaining: 23.6s
6:	learn: 0.5443420	total: 565ms	remaining: 23.6s
7:	learn: 0.5323211	total: 647ms	remaining: 23.6s
8:	learn: 0.5219649	total: 723ms	remaining: 23.4s
9:	learn: 0.5158581	total: 807ms	remaining: 23.4s
10:	learn: 0.5111235	total: 891ms	remaining: 23.4s
11:	learn: 0.5043583	total: 967ms	remaining: 23.2s
12:	learn: 0.5023678	total: 1.05s	remaining: 23.2s
13:	learn: 0.4976994	total: 1.13s	remaining: 23.1s
14:	learn: 0.4937597	total: 1.21s	remaining: 23s
15:	learn: 0.4896269	total: 1.3s	remaining: 23s
16:	learn: 0.4878001	total: 1.38s	remaining: 22.9s
17:	learn: 0.4850109	total: 1.46s	remaining: 22.8s
18:	learn: 0.4835013	total: 1.54s	remaining: 22.8s
19:	learn: 0.4804543	total: 1.62s	remaining: 22

162:	learn: 0.3295853	total: 13.2s	remaining: 11.1s
163:	learn: 0.3269610	total: 13.3s	remaining: 11s
164:	learn: 0.3267153	total: 13.4s	remaining: 11s
165:	learn: 0.3253382	total: 13.5s	remaining: 10.9s
166:	learn: 0.3248743	total: 13.6s	remaining: 10.8s
167:	learn: 0.3237547	total: 13.6s	remaining: 10.7s
168:	learn: 0.3220429	total: 13.7s	remaining: 10.6s
169:	learn: 0.3209481	total: 13.8s	remaining: 10.6s
170:	learn: 0.3190483	total: 13.9s	remaining: 10.5s
171:	learn: 0.3187587	total: 14s	remaining: 10.4s
172:	learn: 0.3179102	total: 14s	remaining: 10.3s
173:	learn: 0.3170628	total: 14.1s	remaining: 10.2s
174:	learn: 0.3167853	total: 14.2s	remaining: 10.1s
175:	learn: 0.3153965	total: 14.3s	remaining: 10.1s
176:	learn: 0.3150187	total: 14.4s	remaining: 9.98s
177:	learn: 0.3142131	total: 14.4s	remaining: 9.9s
178:	learn: 0.3127452	total: 14.5s	remaining: 9.82s
179:	learn: 0.3125558	total: 14.6s	remaining: 9.74s
180:	learn: 0.3112709	total: 14.7s	remaining: 9.66s
181:	learn: 0.3103619

22:	learn: 0.4703662	total: 1.79s	remaining: 21.6s
23:	learn: 0.4683800	total: 1.87s	remaining: 21.6s
24:	learn: 0.4659699	total: 1.95s	remaining: 21.5s
25:	learn: 0.4636859	total: 2.04s	remaining: 21.5s
26:	learn: 0.4632138	total: 2.11s	remaining: 21.4s
27:	learn: 0.4619904	total: 2.19s	remaining: 21.3s
28:	learn: 0.4612569	total: 2.27s	remaining: 21.3s
29:	learn: 0.4606986	total: 2.36s	remaining: 21.2s
30:	learn: 0.4603245	total: 2.44s	remaining: 21.1s
31:	learn: 0.4588531	total: 2.52s	remaining: 21.1s
32:	learn: 0.4583470	total: 2.6s	remaining: 21.1s
33:	learn: 0.4576440	total: 2.68s	remaining: 21s
34:	learn: 0.4567743	total: 2.76s	remaining: 20.9s
35:	learn: 0.4555104	total: 2.85s	remaining: 20.9s
36:	learn: 0.4549609	total: 2.92s	remaining: 20.8s
37:	learn: 0.4527532	total: 3.01s	remaining: 20.7s
38:	learn: 0.4511500	total: 3.09s	remaining: 20.7s
39:	learn: 0.4500649	total: 3.17s	remaining: 20.6s
40:	learn: 0.4496447	total: 3.25s	remaining: 20.5s
41:	learn: 0.4485193	total: 3.33s	

183:	learn: 0.2794479	total: 14.8s	remaining: 9.32s
184:	learn: 0.2782682	total: 14.9s	remaining: 9.24s
185:	learn: 0.2774265	total: 14.9s	remaining: 9.16s
186:	learn: 0.2765159	total: 15s	remaining: 9.08s
187:	learn: 0.2756973	total: 15.1s	remaining: 9s
188:	learn: 0.2749136	total: 15.2s	remaining: 8.92s
189:	learn: 0.2746734	total: 15.3s	remaining: 8.84s
190:	learn: 0.2737932	total: 15.4s	remaining: 8.76s
191:	learn: 0.2728795	total: 15.4s	remaining: 8.68s
192:	learn: 0.2709374	total: 15.5s	remaining: 8.6s
193:	learn: 0.2706902	total: 15.6s	remaining: 8.52s
194:	learn: 0.2688258	total: 15.7s	remaining: 8.44s
195:	learn: 0.2680582	total: 15.8s	remaining: 8.36s
196:	learn: 0.2667960	total: 15.8s	remaining: 8.28s
197:	learn: 0.2654655	total: 15.9s	remaining: 8.2s
198:	learn: 0.2637974	total: 16s	remaining: 8.12s
199:	learn: 0.2626260	total: 16.1s	remaining: 8.04s
200:	learn: 0.2618574	total: 16.2s	remaining: 7.96s
201:	learn: 0.2607374	total: 16.2s	remaining: 7.88s
202:	learn: 0.2595768

43:	learn: 0.4468695	total: 3.49s	remaining: 20.3s
44:	learn: 0.4458195	total: 3.57s	remaining: 20.2s
45:	learn: 0.4454952	total: 3.65s	remaining: 20.1s
46:	learn: 0.4451345	total: 3.69s	remaining: 19.8s
47:	learn: 0.4446997	total: 3.77s	remaining: 19.8s
48:	learn: 0.4440087	total: 3.85s	remaining: 19.7s
49:	learn: 0.4434778	total: 3.93s	remaining: 19.6s
50:	learn: 0.4423189	total: 4.01s	remaining: 19.6s
51:	learn: 0.4411752	total: 4.09s	remaining: 19.5s
52:	learn: 0.4400499	total: 4.17s	remaining: 19.4s
53:	learn: 0.4395975	total: 4.26s	remaining: 19.4s
54:	learn: 0.4394817	total: 4.34s	remaining: 19.3s
55:	learn: 0.4382525	total: 4.42s	remaining: 19.3s
56:	learn: 0.4375319	total: 4.5s	remaining: 19.2s
57:	learn: 0.4358672	total: 4.58s	remaining: 19.1s
58:	learn: 0.4352430	total: 4.66s	remaining: 19s
59:	learn: 0.4331395	total: 4.74s	remaining: 19s
60:	learn: 0.4323092	total: 4.82s	remaining: 18.9s
61:	learn: 0.4318258	total: 4.9s	remaining: 18.8s
62:	learn: 0.4300587	total: 4.98s	rem

205:	learn: 0.2690903	total: 17.2s	remaining: 7.84s
206:	learn: 0.2676639	total: 17.3s	remaining: 7.75s
207:	learn: 0.2663101	total: 17.3s	remaining: 7.67s
208:	learn: 0.2655496	total: 17.4s	remaining: 7.59s
209:	learn: 0.2640873	total: 17.5s	remaining: 7.5s
210:	learn: 0.2633082	total: 17.6s	remaining: 7.42s
211:	learn: 0.2626143	total: 17.7s	remaining: 7.34s
212:	learn: 0.2619179	total: 17.8s	remaining: 7.26s
213:	learn: 0.2610194	total: 17.9s	remaining: 7.17s
214:	learn: 0.2600061	total: 17.9s	remaining: 7.09s
215:	learn: 0.2588663	total: 18s	remaining: 7.01s
216:	learn: 0.2583747	total: 18.1s	remaining: 6.92s
217:	learn: 0.2576706	total: 18.2s	remaining: 6.84s
218:	learn: 0.2571152	total: 18.3s	remaining: 6.76s
219:	learn: 0.2563020	total: 18.4s	remaining: 6.69s
220:	learn: 0.2549247	total: 18.5s	remaining: 6.62s
221:	learn: 0.2537890	total: 18.7s	remaining: 6.55s
222:	learn: 0.2514252	total: 18.8s	remaining: 6.48s
223:	learn: 0.2500603	total: 18.8s	remaining: 6.39s
224:	learn: 0.2

65:	learn: 0.4378419	total: 7.47s	remaining: 26.5s
66:	learn: 0.4374183	total: 7.57s	remaining: 26.3s
67:	learn: 0.4355720	total: 7.68s	remaining: 26.2s
68:	learn: 0.4341917	total: 7.79s	remaining: 26.1s
69:	learn: 0.4337763	total: 7.9s	remaining: 26s
70:	learn: 0.4329004	total: 8.01s	remaining: 25.8s
71:	learn: 0.4319375	total: 8.12s	remaining: 25.7s
72:	learn: 0.4314282	total: 8.23s	remaining: 25.6s
73:	learn: 0.4307824	total: 8.36s	remaining: 25.5s
74:	learn: 0.4296520	total: 8.51s	remaining: 25.5s
75:	learn: 0.4291636	total: 8.62s	remaining: 25.4s
76:	learn: 0.4288797	total: 8.73s	remaining: 25.3s
77:	learn: 0.4278577	total: 8.84s	remaining: 25.2s
78:	learn: 0.4275478	total: 8.95s	remaining: 25s
79:	learn: 0.4272378	total: 9.06s	remaining: 24.9s
80:	learn: 0.4268346	total: 9.16s	remaining: 24.8s
81:	learn: 0.4262940	total: 9.27s	remaining: 24.6s
82:	learn: 0.4257588	total: 9.38s	remaining: 24.5s
83:	learn: 0.4254328	total: 9.49s	remaining: 24.4s
84:	learn: 0.4247373	total: 9.6s	rem

225:	learn: 0.3247322	total: 24.9s	remaining: 8.15s
226:	learn: 0.3245280	total: 25s	remaining: 8.04s
227:	learn: 0.3237916	total: 25.1s	remaining: 7.93s
228:	learn: 0.3227506	total: 25.2s	remaining: 7.82s
229:	learn: 0.3226098	total: 25.3s	remaining: 7.71s
230:	learn: 0.3225389	total: 25.4s	remaining: 7.6s
231:	learn: 0.3221235	total: 25.5s	remaining: 7.49s
232:	learn: 0.3209197	total: 25.7s	remaining: 7.38s
233:	learn: 0.3204063	total: 25.8s	remaining: 7.27s
234:	learn: 0.3201787	total: 25.9s	remaining: 7.16s
235:	learn: 0.3198089	total: 26s	remaining: 7.04s
236:	learn: 0.3196461	total: 26.1s	remaining: 6.93s
237:	learn: 0.3189747	total: 26.2s	remaining: 6.82s
238:	learn: 0.3180926	total: 26.3s	remaining: 6.71s
239:	learn: 0.3173053	total: 26.4s	remaining: 6.6s
240:	learn: 0.3163983	total: 26.5s	remaining: 6.49s
241:	learn: 0.3157015	total: 26.6s	remaining: 6.38s
242:	learn: 0.3152050	total: 26.7s	remaining: 6.27s
243:	learn: 0.3151070	total: 26.8s	remaining: 6.16s
244:	learn: 0.3143

GridSearchCV(cv=3, error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x7fc29826fe10>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'depth': [7], 'learning_rate': [0.1], 'l2_leaf_reg': [4], 'iterations': [300], 'scale_pos_weight': [18]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [25]:
pred = cb_model.predict(x_test)
accuracy_score(pred, y_test), 1-accuracy_score(pred, y_test)

(0.8401318500206015, 0.15986814997939847)

In [26]:
set_test = pd.merge(data,labels_predict,on = "person", how = "right")
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])

In [27]:
preds_kaggle = cb_model.predict_proba(set_test)

In [28]:
preds_kaggle  = preds_kaggle.tolist()
df_kaggel = pd.DataFrame(preds_kaggle)
df_kaggel.head()

,0,1
0,0.871757,0.128243
1,0.336777,0.663223
2,0.724138,0.275862
3,0.995521,0.004479
4,0.772004,0.227996


In [29]:
df_kaggel = pd.DataFrame(df_kaggel[1])                               
df_kaggel["person"] = personas
df_kaggel.rename(columns = {1: 'label'},inplace = True) 
df_kaggel = df_kaggel[['person','label']]

In [30]:
df_kaggel.head()

,person,label
0,00091926,0.128243
1,00091a7a,0.663223
2,000ba417,0.275862
3,000e4d9e,0.004479
4,000e619d,0.227996


In [31]:
df_kaggel["label"].value_counts()

0.000579    2
0.004101    2
0.000306    2
0.000432    2
0.000410    2
0.013767    1
0.017687    1
0.017840    1
0.272379    1
0.749038    1
0.084862    1
0.825504    1
0.196081    1
0.117466    1
0.013288    1
0.052323    1
0.837035    1
0.506580    1
0.169061    1
0.224108    1
0.009307    1
0.699658    1
0.216485    1
0.582163    1
0.784982    1
0.145321    1
0.257374    1
0.191088    1
0.034296    1
0.373958    1
           ..
0.740702    1
0.090072    1
0.081333    1
0.604640    1
0.321604    1
0.057915    1
0.048366    1
0.096520    1
0.172028    1
0.037150    1
0.334052    1
0.013865    1
0.095297    1
0.535950    1
0.072945    1
0.868277    1
0.803031    1
0.456458    1
0.003591    1
0.020323    1
0.355627    1
0.388247    1
0.125410    1
0.929876    1
0.231863    1
0.062842    1
0.050705    1
0.390559    1
0.038954    1
0.851937    1
Name: label, Length: 19410, dtype: int64

In [32]:
df_kaggel.to_csv('Resultados_CatBoost.csv',index=False)